<a href="https://colab.research.google.com/github/AndresT3086/saberPRO-prediction/blob/main/99_modelo_soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# SOLUCIÓN COMPLETA - Predicción Rendimiento Saber Pro
# Este notebook contiene el proceso completo desde la carga de datos hasta la generación del archivo de submission

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 1. CARGAR LOS DATOS
print('='*60)
print('PASO 1: CARGA DE DATOS')
print('='*60)

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print(f'Train: {train.shape}')
print(f'Test: {test.shape}')
print(f'\nColumnas: {train.shape[1]}')
print(f'Estudiantes en train: {len(train)}')
print(f'Estudiantes en test: {len(test)}')

PASO 1: CARGA DE DATOS
Train: (692500, 21)
Test: (296786, 20)

Columnas: 21
Estudiantes en train: 692500
Estudiantes en test: 296786


In [ ]:
# 2. EXPLORACIÓN RÁPIDA
print('\n' + '='*60)
print('PASO 2: EXPLORACIÓN DE DATOS')
print('='*60)

print('\nDistribución de clases:')
print(train['RENDIMIENTO_GLOBAL'].value_counts())
print('\nPorcentajes:')
print(train['RENDIMIENTO_GLOBAL'].value_counts(normalize=True) * 100)

print(f'\nValores faltantes en train: {train.isnull().sum().sum()}')
print(f'Valores faltantes en test: {test.isnull().sum().sum()}')


PASO 2: EXPLORACIÓN DE DATOS

Distribución de clases:
RENDIMIENTO_GLOBAL
alto          175619
bajo          172987
medio-bajo    172275
medio-alto    171619
Name: count, dtype: int64

Porcentajes:
RENDIMIENTO_GLOBAL
alto          25.360144
bajo          24.980072
medio-bajo    24.877256
medio-alto    24.782527
Name: proportion, dtype: float64

Valores faltantes en train: 297378
Valores faltantes en test: 128614


In [ ]:
# 3. GUARDAR IDs Y SEPARAR VARIABLES
print('\n' + '='*60)
print('PASO 3: PREPARACIÓN DE DATOS')
print('='*60)

test_ids = test['ID'].copy()
y_train = train['RENDIMIENTO_GLOBAL'].copy()
X_train = train.drop(['ID', 'RENDIMIENTO_GLOBAL'], axis=1)
X_test = test.drop(['ID'], axis=1)

print(f'X_train: {X_train.shape}')
print(f'y_train: {y_train.shape}')
print(f'X_test: {X_test.shape}')


PASO 3: PREPARACIÓN DE DATOS
X_train: (692500, 19)
y_train: (692500,)
X_test: (296786, 19)


In [ ]:
# 4. PREPROCESAMIENTO
print('\n' + '='*60)
print('PASO 4: PREPROCESAMIENTO')
print('='*60)

# Identificar tipos de columnas
columnas_numericas = X_train.select_dtypes(include=['int64', 'float64']).columns
columnas_categoricas = X_train.select_dtypes(include=['object']).columns

print(f'Variables numéricas: {len(columnas_numericas)}')
print(f'Variables categóricas: {len(columnas_categoricas)}')

# Imputar valores faltantes en numéricas
for col in columnas_numericas:
    if X_train[col].isnull().sum() > 0:
        mediana = X_train[col].median()
        X_train[col] = X_train[col].fillna(mediana)
        X_test[col] = X_test[col].fillna(mediana)

# Imputar valores faltantes en categóricas
for col in columnas_categoricas:
    if X_train[col].isnull().sum() > 0:
        moda = X_train[col].mode()[0]
        X_train[col] = X_train[col].fillna(moda)
        X_test[col] = X_test[col].fillna(moda)

print('\nValores faltantes imputados')

# One-Hot Encoding
print(f'\nColumnas antes de encoding: {X_train.shape[1]}')
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

# Alinear columnas
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

print(f'Columnas después de encoding: {X_train.shape[1]}')
print('\nPreprocesamiento completado ✓')


PASO 4: PREPROCESAMIENTO
Variables numéricas: 5
Variables categóricas: 14

Valores faltantes imputados

Columnas antes de encoding: 19
Columnas después de encoding: 1028

Preprocesamiento completado ✓


In [ ]:
# 5. VALIDACIÓN CRUZADA
print('\n' + '='*60)
print('PASO 5: VALIDACIÓN DEL MODELO')
print('='*60)

# Dividir para validación
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

print(f'Train: {X_train_split.shape}')
print(f'Validation: {X_val_split.shape}')


PASO 5: VALIDACIÓN DEL MODELO
Train: (554000, 1028)
Validation: (138500, 1028)


In [ ]:
# 6. ENTRENAR MODELO
print('\n' + '='*60)
print('PASO 6: ENTRENAMIENTO DEL MODELO')
print('='*60)
print('Entrenando Random Forest...')

modelo = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=4,
    random_state=42,
    n_jobs=-1
)

modelo.fit(X_train_split, y_train_split)
print('Modelo entrenado ✓')


PASO 6: ENTRENAMIENTO DEL MODELO
Entrenando Random Forest...
Modelo entrenado ✓


In [ ]:
# 7. EVALUAR MODELO
print('\n' + '='*60)
print('PASO 7: EVALUACIÓN DEL MODELO')
print('='*60)

y_pred_val = modelo.predict(X_val_split)
accuracy = accuracy_score(y_val_split, y_pred_val)

print(f'\nAccuracy en validación: {accuracy:.4f} ({accuracy*100:.2f}%)')
print('\nReporte de clasificación:')
print(classification_report(y_val_split, y_pred_val))


PASO 7: EVALUACIÓN DEL MODELO

Accuracy en validación: 0.3966 (39.66%)

Reporte de clasificación:
              precision    recall  f1-score   support

        alto       0.48      0.61      0.54     35124
        bajo       0.40      0.57      0.47     34597
  medio-alto       0.30      0.18      0.23     34324
  medio-bajo       0.32      0.21      0.26     34455

    accuracy                           0.40    138500
   macro avg       0.37      0.40      0.37    138500
weighted avg       0.37      0.40      0.37    138500



In [ ]:
# 8. ENTRENAR MODELO FINAL
print('\n' + '='*60)
print('PASO 8: MODELO FINAL CON TODOS LOS DATOS')
print('='*60)

modelo_final = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=4,
    random_state=42,
    n_jobs=-1
)

modelo_final.fit(X_train, y_train)
print('Modelo final entrenado ✓')


PASO 8: MODELO FINAL CON TODOS LOS DATOS
Modelo final entrenado ✓


In [ ]:
# 9. PREDICCIONES EN TEST
print('\n' + '='*60)
print('PASO 9: PREDICCIONES')
print('='*60)

predicciones = modelo_final.predict(X_test)

print(f'Predicciones realizadas: {len(predicciones)}')
print('\nDistribución de predicciones:')
print(pd.Series(predicciones).value_counts().sort_index())


PASO 9: PREDICCIONES
Predicciones realizadas: 296786

Distribución de predicciones:
alto           96248
bajo          108489
medio-alto     43926
medio-bajo     48123
Name: count, dtype: int64


In [ ]:
# 10. CREAR ARCHIVO SUBMISSION
print('\n' + '='*60)
print('PASO 10: GENERAR ARCHIVO DE SUBMISSION')
print('='*60)

submission = pd.DataFrame({
    'ID': test_ids,
    'RENDIMIENTO_GLOBAL': predicciones
})

submission.to_csv('submission.csv', index=False)

print(f'\nArchivo creado: submission.csv')
print(f'Total de predicciones: {len(submission)}')
print('\nPrimeras 10 predicciones:')
print(submission.head(10))

print('\n' + '='*60)
print('✅ PROCESO COMPLETADO')
print('='*60)
print('Descarga submission.csv y súbelo a Kaggle')


PASO 10: GENERAR ARCHIVO DE SUBMISSION

Archivo creado: submission.csv
Total de predicciones: 296786

Primeras 10 predicciones:
       ID RENDIMIENTO_GLOBAL
0  550236               alto
1   98545         medio-alto
2  499179               alto
3  782980               bajo
4  785185               bajo
5   58495               bajo
6  705444               alto
7  557548               alto
8  519909               bajo
9  832058               alto

✅ PROCESO COMPLETADO
Descarga submission.csv y súbelo a Kaggle
